# CO2 Emissions Prediction 
## Objetivo
Extrair indicadores do World Development Indicators (WDI) via API do Banco Mundial e gerar:
- data/raw/wdi_long.csv (formato longo: país–indicador–ano–valor)
- data/processed/wdi_wide.csv (formato wide estilo Kaggle: 1 linha por país+indicador, colunas por ano)

In [ ]:
import logging
from pathlib import Path

import requests
import pandas as pd

logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")
logger = logging.getLogger("wdi_etl")

In [ ]:
# Cria session com retry/backoff
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def build_session() -> requests.Session:
    """
    Session com retry/backoff para tornar o ETL robusto a:
    - timeouts intermitentes
    - 429/5xx do servidor
    """
    session = requests.Session()

    retry = Retry(
        total=8,
        connect=8,
        read=8,
        backoff_factor=1.2,  # 1.2s, 2.4s, 4.8s...
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET"],
        raise_on_status=False,
        respect_retry_after_header=True,
    )

    adapter = HTTPAdapter(max_retries=retry, pool_connections=10, pool_maxsize=10)
    session.mount("https://", adapter)
    session.mount("http://", adapter)
    return session

SESSION = build_session()

## Configurações do ETL
- INDICATORS: lista dos códigos WDI (fonte oficial)
- YEARS: intervalo 2000–2020
- PATH_LONG/PATH_WIDE: saídas em disco

In [ ]:
INDICATORS = [
    "EG.ELC.ACCS.ZS",       # Access to electricity (% of population)
    "AG.LND.AGRI.ZS",       # Agricultural land (% of land area)
    "ER.H2O.FWVT.ZS",       # Annual freshwater withdrawals, total (% of internal resources)
    "AG.LND.ARBL.ZS",       # Arable land (% of land area)
    "AG.LND.FRST.ZS",       # Forest area (% of land area)
    "EG.USE.ELEC.KH.PC",    # Electric power consumption (kWh per capita)
    "EG.USE.PCAP.KG.OE",    # Energy use (kg of oil equivalent per capita)
    "EG.ELC.RNEW.ZS",       # Renewable electricity output (% of total electricity output)
    "EG.FEC.RNEW.ZS",       # Renewable energy consumption (% of total final energy consumption)
    "SP.POP.GROW",          # Population growth (annual %)
    "NY.GDP.PCAP.CD",       # GDP per capita (current US$)
    "EN.ATM.CO2E.PC",       # CO2 emissions (metric tons per capita)
]

YEARS = list(range(2000, 2021))
DATE_RANGE = f"{min(YEARS)}:{max(YEARS)}"

BASE = "https://api.worldbank.org/v2"

PATH_LONG = Path("data/raw/wdi_long.csv")
PATH_WIDE = Path("data/processed/wdi_wide.csv")

In [ ]:
# Criar pastas
Path("data/raw").mkdir(parents=True, exist_ok=True)
Path("data/processed").mkdir(parents=True, exist_ok=True)
logger.info("Pastas data/raw e data/processed prontas.")

## Função utilitária: paginação da APIA 
API do World Bank é paginada. A função abaixo:
- percorre todas as páginas
- acumula os itens
- retorna uma lista única

In [ ]:
# Função para capturar todas as paginas
import time

def wb_get_all_pages(url: str, params: dict, session: requests.Session = SESSION) -> list:
    """
    Busca todas as páginas de um endpoint do World Bank (API v2),
    com resiliência a timeouts e respostas lentas.
    """
    params = dict(params)
    params.setdefault("format", "json")
    params.setdefault("per_page", 2000)  # menor = resposta mais leve e menos timeout

    page = 1
    out = []

    while True:
        params["page"] = page

        # timeout pode ser tupla: (connect_timeout, read_timeout)
        resp = session.get(url, params=params, timeout=(10, 180))
        resp.raise_for_status()

        data = resp.json()

        if not isinstance(data, list) or len(data) < 2 or data[1] is None:
            break

        meta, items = data[0], data[1]
        out.extend(items)

        pages = int(meta.get("pages", 1))
        if page >= pages:
            break

        page += 1

        # “educado” com a API (ajuda estabilidade)
        time.sleep(0.2)

    return out

## Países e filtro de agregados
Baixamos o catálogo de países para:
- identificar e remover “Aggregates” (World, regiões, etc.)
- manter apenas países/territórios como unidades de análise

In [ ]:
# Baixar países + valid_country_codes
countries_url = f"{BASE}/country"
countries_items = wb_get_all_pages(countries_url, params={"per_page": 20000})
df_countries = pd.json_normalize(countries_items)

# Agregados normalmente aparecem com region.value == "Aggregates"
if "region.value" in df_countries.columns:
    df_countries["is_aggregate"] = df_countries["region.value"].eq("Aggregates")
else:
    df_countries["is_aggregate"] = False  # fallback defensivo

valid_country_codes = set(df_countries.loc[~df_countries["is_aggregate"], "id"].tolist())

logger.info("Total entidades (inclui agregados): %s", df_countries.shape[0])
logger.info("Total países/territórios (sem agregados): %s", len(valid_country_codes))

## Função: download de 1 indicador (formato longo)
Baixa um indicador WDI para todos os países, no intervalo configurado, e devolve:
- Country Code,
- Country Name,
- Indicator Code,
- Indicator Name,
- Year,
- Value.

In [ ]:
# Download de indicador (formato longo)

def download_indicator_long(indicator_code: str) -> pd.DataFrame:
    """
    Baixa um indicador WDI (todos países) para o intervalo DATE_RANGE.

    Retorna um DataFrame no formato longo:
      (Country, Indicator, Year) -> Value
    """
    url = f"{BASE}/country/all/indicator/{indicator_code}"
    items = wb_get_all_pages(url, params={"date": DATE_RANGE})

    rows = []
    for it in items:
        # A API é semi-estruturada; usamos .get com fallback seguro
        ccode = (it.get("country") or {}).get("id")
        cname = (it.get("country") or {}).get("value")
        icode = (it.get("indicator") or {}).get("id")
        iname = (it.get("indicator") or {}).get("value")
        year = it.get("date")
        val = it.get("value")

        if ccode is None or year is None:
            continue

        rows.append({
            "Country Code": ccode,
            "Country Name": cname,
            "Indicator Code": icode,
            "Indicator Name": iname,
            "Year": int(year),
            "Value": val,
        })

    df = pd.DataFrame(rows)

    # Normaliza tipos (Value pode vir None)
    if not df.empty:
        df["Value"] = pd.to_numeric(df["Value"], errors="coerce")
        df["Year"] = df["Year"].astype(int)

    return df

# Construção do dataset LONG (com cache)
- Se data/raw/wdi_long.csv existir → carrega
- Caso contrário → baixa todos os indicadores e salva


In [ ]:
# Construir df_long com cache
if PATH_LONG.exists():
    df_long = pd.read_csv(PATH_LONG)
    df_long["Year"] = df_long["Year"].astype(int)
    df_long["Value"] = pd.to_numeric(df_long["Value"], errors="coerce")
    logger.info("Carregado do cache: %s | shape=%s", PATH_LONG, df_long.shape)
else:
    dfs = []
    for i, ind in enumerate(INDICATORS, start=1):
        logger.info("[%s/%s] Baixando indicador: %s", i, len(INDICATORS), ind)
        dfs.append(download_indicator_long(ind))

    df_long = pd.concat(dfs, ignore_index=True)
    df_long = df_long[df_long["Year"].between(min(YEARS), max(YEARS))]

    df_long.to_csv(PATH_LONG, index=False)
    logger.info("Salvo: %s | shape=%s", PATH_LONG, df_long.shape)